# Functions used for all the jupyter notebook files

import necessary library's

In [2]:
import pandas as pd
import numpy as np
import urllib.parse
import math
from sklearn.metrics import jaccard_score
import random

In [5]:
def jaccard_similarity(active_user_series, other):
    '''
    calculate similary between two users
    '''
    intersection = np.logical_and(active_user_series, other)
    union = np.logical_or(active_user_series, other)
    return np.sum(intersection) / np.sum(union)

In [3]:
def GetIDFromName(name):
    '''
    Get the ID from name
    '''
    data = pd.read_csv('ID_name_group_type_2023.csv', index_col=0)
    name_df=data[data['name']==name]
    ID = name_df['ID'].iloc[0]
    return ID

In [ ]:
def GetNameFromID(ID):
    '''
    Get the name from a ID
    '''
    data = pd.read_csv('ID_name_group_type_2023.csv', index_col=0)
    ID_df = data[data['ID']==ID]
    name = ID_df['name'].iloc[0]
    return name

In [4]:
def GetTypeFromName(name):
    '''
    Get the type from a name
    '''
    data = pd.read_csv('ID_name_group_type_2023.csv', index_col=0)
    name_df = data[data['name']==name]
    type = name_df['type'].iloc[0]
    return type

In [5]:
def GetTypeFromID(ID):
    '''
    Get the type from an ID
    '''
    data = pd.read_csv('ID_name_group_type_2023.csv', index_col=0)
    ID_df = data[data['ID']==ID]
    type = ID_df['type'].iloc[0]
    return type

In [8]:
def GetMemberItems(df, member):
    '''
    Get the community's a user is member from the dataframe that is given as parameter
    '''
    all_items = []
    
    df_small = df[df.index == member]
    for i in range(len(df_small.columns)):
        if df_small[df_small.columns[i]].iloc[0] == 1:

            all_items.append(df_small.columns[i]) 
    
 
    return all_items   

In [13]:
def lastWord(string):
    '''
    Get the last word of a string
    '''
    lis = list(string.split(" "))
    length = len(lis)
    return lis[length-1]

In [15]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

### The function to create recommendations

In [ ]:
def Recommendation(user_ID, t=0.175, top_n=5):
    '''
    Recommendation returns: user_ID, user_items, selected neighbors and recommended community's
    
    The model used to make a recommendation is user-based collaborative filtering which works in 3 steps:
        1) Calculate similarity with active users and all other users based on membership
        2) Select amount of similar users, the neighbors
        3) Calculate the weighted sum for each community and recommened the top_n community's
    
    The parameters:
        - t:     the amount of similar users you want to select. Default is 0.175
        - top_n: the amount of recommendations you would like. Default is 5
        !note it is possible that you don't have any recommendations or the actual amount doesn't meet your wishes
    '''

    df = pd.read_csv('Prediction_matrix_2023', index_col=0)
    all_columns = list(df.columns)
    
    #Step1) Get active user information
    user_dict = dict.fromkeys(all_columns, 0) 
    user_items = GetMemberItems(df, user_ID)
    for item in user_items:
        user_dict[item] = 1
    active_user = pd.DataFrame(data=[user_dict], columns=all_columns, index=[user_ID])
    active_user_series = active_user.iloc[0]
    other_users = df.drop(user_ID)
    
    #Step2) Calculate similarity between active user and all other users
    results = other_users.apply(lambda row: jaccard_similarity(active_user_series, row), axis=1) #calculate similarities

    results = results.sort_values(ascending=False)
    neighbors = results[(results.values > t) & (results.values < 1)].index
    scores = results[(results.values > t) & (results.values < 1)].values
    
    #Step3) calculate weighted sum for each community
    df_neighbors = df.loc[neighbors] 
    df_neighbors['weights'] = scores
    prediction_score = {}
    for w in range(len(df_neighbors.columns)-1): 
        if sum(df_neighbors['weights']) == 0:
            prediction_score[df_neighbors.columns[w]] = 0
        else:
            score = np.average(df_neighbors[df_neighbors.columns[w]], weights=df_neighbors['weights']) 
            prediction_score[df_neighbors.columns[w]] = score

    recommended_items = []

    prediction_sorted = dict(sorted(prediction_score.items(), key=lambda item: item[1], reverse=True))
    amount=0
    recommended = []
    
    #Step4) recommend top_n community's if not a hub, not in user_items and score higher then 0
    for f in prediction_sorted.keys(): 
        if (GetTypeFromID(f)!='hub') and (f not in user_items) and prediction_sorted[f] > 0:
            amount+=1
            recommended.append(f)
        if amount > (top_n-1):
            break
    
    return user_ID, user_items, neighbors, recommended



### The function to graph the user, community's, neighbors and recommended community's

In [ ]:
def MakeVisual(user_ID, user_items, neighbors, recommended):
    '''
    MakeVisual returns: graph with the active user, its community's, the selected neighbors and the recommended community's
    The graph is saved as: graph.html
    
    The parameters:
        - user_ID:     the ID of the active users. This is also return by the Recommendation function.
        - user_items:  the community's of which the active user is member of. Return by Recommendation function.
        - neighbors:   the selected neighbors which shows most similarity with active user. Return by Recommendation function.
        - recommended: the recommended community's to the active user. Return by Recommendation function.
    '''
    
    df_open = pd.read_csv('data_graph_matrix_2023.csv', index_col=0)
    df_select = df_open.loc[:,:]
    neighbors = neighbors.append(pd.Index([user_ID]))
    
    for a in range(len(df_select)-1,-1,-1): #select right items and users
        if df_select['source'].iloc[a] not in neighbors and df_select['target'].iloc[a] not in neighbors:
            df_select = df_select.drop(df_open.index[a])
            
    for b in range(len(df_select)): #add correct labels
        if df_select['target'].iloc[b] == user_ID:
            df_select['type Target'].iloc[b] = 'Active User'
        elif df_select['source'].iloc[b] == user_ID:
            df_select['type Source'].iloc[b] = 'Active User'

        elif df_select['target'].iloc[b] in neighbors[0:-1]:
            df_select['type Target'].iloc[b] = 'Selected Neighbor'
        elif df_select['source'].iloc[b] in neighbors[0:-1]:
            df_select['type Source'].iloc[b] = 'Selected Neighbor'

        if df_select['target'].iloc[b] in recommended:
            df_select['type Target'].iloc[b] = 'Recommended Community'
        elif df_select['source'].iloc[b] in recommended:
            df_select['type Source'].iloc[b] = 'Recommended Community'

        if df_select['target'].iloc[b] in user_items :
            df_select['type Target'].iloc[b] = 'Your Community'
        elif df_select['source'].iloc[b] in user_items :
            df_select['type Source'].iloc[b] = 'Your Community'
        
    OPEN_ID = nx.Graph()

    nodes = list(set(df_select['name Source']).union(set(df_select['name Target']))) #adding nodes
    for node in nodes:
        category1_matches = df_select['name Source'] == node
        category2_matches = df_select['name Target'] == node
        if category1_matches.any() or category2_matches.any():
            category1 = df_select.loc[category1_matches, 'type Source'].iloc[0] if category1_matches.any() else None
            category2 = df_select.loc[category2_matches, 'type Target'].iloc[0] if category2_matches.any() else None
            OPEN_ID.add_node(node, category1=category1, category2=category2)

    edges = [(row['name Source'], row['name Target']) for index, row in df_select.iterrows()] #adding edges
    OPEN_ID.add_edges_from(edges)
    colors = {'Active User':"red", 'Selected Neighbor':'blue', 'Your Community':'orange', 'Recommended Community':'magenta', 'hub':'black', 'challenge':'black', 'opportunity':'black', 'user':'black', 'organization':'black'}
    node_colors = [colors[OPEN_ID.nodes[node]['category1']] if OPEN_ID.nodes[node]['category1'] else colors[OPEN_ID.nodes[node]['category2']] for node in OPEN_ID.nodes()]

    nt = Network(height="1000px", width="1000px") #create network
    nt.barnes_hut(gravity=-2000, central_gravity=1.2, spring_length=40, spring_strength=0, damping=0.09, overlap=0) #makes sure you can move the nodes
    for i in range(len(nodes)): #add nodes to Network
        nt.add_node(nodes[i], label=nodes[i], value=20, color=node_colors[i], font='25px arial black')

    for x in range(len(edges)): #add edges to Network
        nt.add_edge(edges[x][0], edges[x][1])

    legend_items = [] # Create the legend
    other_communities = set()
    for category, color in colors.items():
        if color == 'black':
            other_communities.add(category)
        else:
            legend_items.append((category, color))

    if other_communities:
        legend_items.append(('Other Communities', 'black'))

    html_file = "graph.html"
    nt.save_graph(html_file)

    title_html = f'<h1 style="text-align: center;">Connectivity graph of {GetNameFromID(user_ID)}</h1>'
    legend_html = '<div style="position: absolute; top: 100px; left: 1025px; font-family: Arial, sans-serif; z-index: 9999;">'
    legend_html += '<h2 style="text-align: left;">Legend</h2>'
    for category, color in legend_items:
        legend_html += f'<div style="display: flex; align-items: center; margin-bottom: 5px;">' \
                       f'<div style="width: 20px; height: 20px; background-color: {color}; margin-right: 10px;"></div>' \
                       f'<span style="font-size: 14px;">{category}</span>' \
                       f'</div>'
    legend_html += '</div>'

    with open(html_file, 'r+') as file:
        content = file.read()
        content = content.replace('<body>', f'<body>{title_html}{legend_html}')
        file.seek(0)
        file.write(content)
        file.truncate()

    webbrowser.open_new_tab(html_file)